In [1]:
import os
import sys

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)

from actionweaver.llms import patch
from langchain_community.utilities.github import GitHubAPIWrapper
from langsmith.run_helpers import traceable


from bot import AutoCoder
from rag import RepositoryIndex



github_repository = "TengHu/auto_coder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)

index = RepositoryIndex(github_api, github_repository)


Indexing codebase TengHu/auto_coder
Indexing bot.py
Indexing rag.py


In [2]:
auto_coder = AutoCoder(github_api, index)
res = auto_coder("What are the open issues?")

[System] Branch 'aw_demo_bot_v1' created successfully, and set as current active branch.
There are currently no open issues in the GitHub repository.


In [ ]:
print(res)